### Simple Gen AI app using langchain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

In [4]:
## Data ingestion-- from the website we need to scrape the data 
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier")

In [6]:
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='\n\n\n\n\nOptimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundPromptsConceptual GuideDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referencePrompt EngineeringTutorialsOptimize a classifierOn this pageOptimize a classifier\nThis tutorial walks through 

In [8]:
## divide text into chunks 
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='Optimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundPromptsConceptual GuideDeployment (LangGraph Platform)Admin

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [10]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [11]:
vectorstoredb

In [14]:
## Query from  a vector store db
query="We can now set up automations to move examples with feedback of some"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'In this example the model will classify it as a bug, whereas I really want this to be classified as documentation.\nls_client = Client()run_id = uuid.uuid4()topic_classifier(    "fix bug in documentation",    langsmith_extra={"run_id": run_id})ls_client.create_feedback(    run_id,    key="correction",    correction="documentation")\nSet up automationsâ€‹\nWe can now set up automations to move examples with feedback of some form into a dataset.\nWe will set up two automations, one for positive feedback and the other for negative feedback.\nThe first will take all runs with positive feedback and automatically add them to a dataset.\nThe logic behind this is that any run with positive feedback we can use as a good example in future iterations.\nLet\'s create a dataset called classifier-github-issues to add this data to.'

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [18]:
## Retrieval Chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
Answer the following questions based only on the provided context:
<context>
{context}
</context>
"""
)
document_chain= create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000027FBF7B5B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000027FBF79E910>, root_client=<openai.OpenAI object at 0x0000027FBF548B90>, root_async_client=<openai.AsyncOpenAI object at 0x0000027FBFA2C8D0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docum

In [19]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"We can now set up automations to move examples with feedback of some",
    "context":[Document(page_content="We can now set up automations to move examples with feedback of some")]
})


"I'm sorry, but the provided context is incomplete, and I need more information to answer any questions related to it. If you can provide additional context or details, I would be happy to help."

In [28]:
## input---> Retriever -----> vectorstoredb
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain) 

In [29]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027F8141E850>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Cha

In [32]:
### Get the response from llm
response=retrieval_chain.invoke({"input":"We can now set up automations to move examples with feedback of some"})
response['answer']

'1. What is the main objective of the example provided in the context?\n\n   The main objective of the example is to build a bot that classifies GitHub issues based on their title and collects user feedback to improve the classifier\'s performance.\n\n2. What is the purpose of setting up automations in the context?\n\n   The purpose of setting up automations is to move examples with feedback into a dataset, with one automation for positive feedback and another for negative feedback. This helps in using runs with positive feedback as good examples for future iterations.\n\n3. What dataset is mentioned for storing data with positive feedback?\n\n   The dataset mentioned for storing data with positive feedback is called "classifier-github-issues."\n\n4. What environment variable is set to send all traces to a specific project?\n\n   The environment variable set to send all traces to a specific project is "LANGCHAIN_PROJECT," with the value "classifier."'

In [33]:
response

{'input': 'We can now set up automations to move examples with feedback of some',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='In this example the model will classify it as a bug, whereas I really want this to be classified as documentation.\nls_client = Client()run_id = uuid.uuid4()topic_classifier(    "fix bug in documentation",    langsmith_extra={"run_id": run_id})ls_client.create_feedback(    run_id,    key="correction",    correction="documentation")\nSet up automationsâ€‹\nWe can now set up automations to move examples with feedback of some form into a dataset.\nWe will set up two automations, one for positive feedback and the other for negative feedback.\nThe first will take all runs with positive feedback a